In [0]:
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType
import time
import os

schema_cliente = StructType([
    StructField("cliente_id", StringType(), True),
    StructField("nome", StringType(), True), 
    StructField("estado", StringType(), True),
    StructField("data_criacao", TimestampType(), True),
])

bronze = "/Volumes/workspace/bravium/bronze/cliente/"
silver = "/Volumes/workspace/bravium/silver/cliente/"

In [0]:
df_stream = spark.readStream\
    .format("delta")\
    .load(bronze)

df_stream = (
    df_stream.dropDuplicates(["cliente_id", 'data_criacao'])
             .withWatermark("data_criacao", "10 minutes")
)

In [0]:
def upsert_to_delta_sql(batch_df, batch_id):
    batch_df.createOrReplaceTempView("staging")
    spark.read.format("delta").load(silver).createOrReplaceTempView("silver")

    spark.sql("""
        MERGE INTO silver t
        USING staging s
        ON t.cliente_id = s.cliente_id
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)


(
    df_stream.writeStream
    .foreachBatch(upsert_to_delta)
    .option("checkpointLocation", "/Volumes/workspace/bravium/silver/checkpoints/cliente")
    .outputMode("update")
    .trigger(availableNow=True)
    .start()
    .awaitTermination()
)

25/09/28 21:48:22 Listener thread still active, was not able to reconnect
